In [1]:
import pandas as pd

energy = pd.read_csv("data/energy_use_per_person.csv")
economy = pd.read_csv("data/gini.csv")
health = pd.read_csv("data/life_expectancy_years.csv")
education = pd.read_csv("data/owid_education_idx.csv")

In [2]:
energy = energy.dropna(axis=1, thresh=100)
energy = energy.dropna(axis=0, thresh=44).reset_index(drop=True)
energy["2014"] = energy["2014"].fillna(energy["2013"])

education_1970 = education["1970"]
education = education[energy.columns].copy()
def fill(df, year, data_year=None):
    years = [1, 2, 3, 4]
    if type(data_year) != type(pd.Series(dtype=int)):
        data_year = df[str(year)]
    for y in years:
        df[str(y+year)] = data_year
    return df
education = fill(education.copy(), 1970, education_1970)
for year in [1975, 1980, 1985]:
    education = fill(education.copy(), year)
education = education.loc[education["country"].isin(energy["country"])].dropna(axis=0).reset_index(drop=True)
education = education.set_index('country').stack().reset_index().rename(columns={'level_1': 'Year', 0: 'value'})
education['theme'] = 'education'
education

,country,Year,value,theme
0,Albania,1971,0.221,education
1,Albania,1972,0.221,education
2,Albania,1973,0.221,education
3,Albania,1974,0.221,education
4,Albania,1975,0.265,education
...,...,...,...,...
4439,Zimbabwe,2010,0.487,education
4440,Zimbabwe,2011,0.487,education
4441,Zimbabwe,2012,0.527,education
4442,Zimbabwe,2013,0.533,education


In [3]:
energy = energy.loc[energy["country"].isin(education["country"].unique())].reset_index(drop=True)
energy_cols = energy.columns
energy_country = energy["country"]
energy = energy.set_index('country').stack().reset_index().rename(columns={'level_1': 'Year', 0: 'value'})
energy['theme'] = 'energy'
energy

,country,Year,value,theme
0,Albania,1971,785.0,energy
1,Albania,1972,866.0,energy
2,Albania,1973,763.0,energy
3,Albania,1974,777.0,energy
4,Albania,1975,827.0,energy
...,...,...,...,...
4439,Zimbabwe,2010,745.0,energy
4440,Zimbabwe,2011,795.0,energy
4441,Zimbabwe,2012,824.0,energy
4442,Zimbabwe,2013,845.0,energy


In [4]:
economy = economy[energy_cols]
economy = economy.loc[economy["country"].isin(energy_country)].reset_index(drop=True)
economy = economy.set_index('country').stack().reset_index().rename(columns={'level_1': 'Year', 0: 'value'})
economy['theme'] = 'economy'
economy

,country,Year,value,theme
0,Albania,1971,26.8,economy
1,Albania,1972,26.8,economy
2,Albania,1973,26.8,economy
3,Albania,1974,26.8,economy
4,Albania,1975,26.8,economy
...,...,...,...,...
4439,Zimbabwe,2010,43.2,economy
4440,Zimbabwe,2011,43.2,economy
4441,Zimbabwe,2012,43.2,economy
4442,Zimbabwe,2013,43.2,economy


In [5]:
health = health[energy_cols]
health = health.loc[health["country"].isin(energy_country)].reset_index(drop=True)
health = health.set_index('country').stack().reset_index().rename(columns={'level_1': 'Year', 0: 'value'})
health['theme'] = 'health'
health

,country,Year,value,theme
0,Albania,1971,67.5,health
1,Albania,1972,68.1,health
2,Albania,1973,68.7,health
3,Albania,1974,69.3,health
4,Albania,1975,69.8,health
...,...,...,...,...
4439,Zimbabwe,2010,49.7,health
4440,Zimbabwe,2011,52.4,health
4441,Zimbabwe,2012,54.9,health
4442,Zimbabwe,2013,56.8,health


In [6]:
df = pd.merge(health[['country', 'Year', 'value']], economy[['country', 'Year', 'value']],  how='left', 
              left_on=['country','Year'], right_on = ['country','Year'])
df = pd.merge(df, energy[['country', 'Year', 'value']],  how='left', left_on=['country','Year'], 
              right_on = ['country','Year'])
df = df.rename(columns={'Year': 'year', 'value_x': 'health', 'value_y': 'economy', 'value': 'energy'})
df = pd.merge(df, education[['country', 'Year', 'value']],  how='left', left_on=['country','year'], 
              right_on = ['country','Year'])
df = df.rename(columns={'value': 'education'})
df = df.drop(columns = 'Year')
df["year"] = df["year"].astype(int)
df.to_csv("data/data_merge.csv", index=False)

In [7]:
df

,country,year,health,economy,energy,education
0,Albania,1971,67.5,26.8,785.0,0.221
1,Albania,1972,68.1,26.8,866.0,0.221
2,Albania,1973,68.7,26.8,763.0,0.221
3,Albania,1974,69.3,26.8,777.0,0.221
4,Albania,1975,69.8,26.8,827.0,0.265
...,...,...,...,...,...,...
4439,Zimbabwe,2010,49.7,43.2,745.0,0.487
4440,Zimbabwe,2011,52.4,43.2,795.0,0.487
4441,Zimbabwe,2012,54.9,43.2,824.0,0.527
4442,Zimbabwe,2013,56.8,43.2,845.0,0.533
